In [2]:
import cv2
from ultralytics import YOLO, solutions

# Cargar el modelo YOLO para pose estimation
model = YOLO("yolov8x-pose.pt")

# Abrir el video de entrenamiento
cap = cv2.VideoCapture("C:/Users/USUARIO/Proyectos_IA/Datasets/videos_gym/press_declinado/correcto/press_declinado_correcto3.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Inicializar el escritor de video
video_writer = cv2.VideoWriter("workouts_press_banca.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Configuración del objeto de conteo, ahora para press de banca
gym_object = solutions.AIGym(
    line_thickness=2,
    view_img=True,
    kpts_to_check=[6, 8, 10, 5, 7, 9],  # Puntos de los brazos para el press de banca
)

# Variables para el conteo de repeticiones
reps_count = 0
down = False

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Detectar poses con YOLO
    results = model.track(im0, verbose=False)
    
    # Verificar si hay resultados y puntos clave detectados
    for result in results:
        if result.keypoints is not None and len(result.keypoints) >= 11:
            keypoints = result.keypoints  # Acceder a los puntos clave del resultado
            
            # Puntos clave del brazo izquierdo (5, 7, 9) y brazo derecho (6, 8, 10)
            shoulder_left, elbow_left, wrist_left = keypoints[5], keypoints[7], keypoints[9]
            shoulder_right, elbow_right, wrist_right = keypoints[6], keypoints[8], keypoints[10]
            
            # Lógica para contar repeticiones
            if elbow_left[1] > shoulder_left[1] and elbow_right[1] > shoulder_right[1]:  # Codos por debajo de los hombros
                down = True
            elif down and elbow_left[1] < shoulder_left[1] and elbow_right[1] < shoulder_right[1]:  # Codos por encima de los hombros
                reps_count += 1
                print(f"Repeticiones: {reps_count}")
                down = False

    # Añadir la lógica de conteo al video
    im0 = gym_object.start_counting(im0, results)
    video_writer.write(im0)

# Cerrar y liberar recursos
cv2.destroyAllWindows()
video_writer.release()

Video frame is empty or video processing has been successfully completed.
